In [ ]:
r = requests.request(method="GET",url="https://netrunnerdb.com/api/2.0/public/decklists/by_date/2023-02-28")

from datetime import date, timedelta
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

import tqdm
from ratelimit import limits, sleep_and_retry

@sleep_and_retry
@limits(calls=1, period=timedelta(seconds=0.25).total_seconds())
def get_decks(date_string, card_dictionary):
    r = requests.request(method="GET",url=f"https://netrunnerdb.com/api/2.0/public/decklists/by_date/{date_string}")

    for decklist in r.json()['data']:
        for cid, count in decklist['cards'].items():
            try:
                card_dictionary[cid]['count'] += count
                card_dictionary[cid]['decklists'] += 1
            except KeyError:
                card_dictionary[cid] = {'count': count, 'decklists':1}
    return card_dictionary

card_dict = {}

# for year in range(2022,2023):
#     start_date = date(year,10,1)
#     end_date = date(year,12,31)
#     for formatted_date in daterange(start_date=start_date, end_date=end_date):
#         get_decks(card_dictionary=card_dict, date_string=formatted_date)

for formatted_date in tqdm.tqdm(daterange(start_date=date(2022,1,1), end_date=date(2023,5,5))):
    get_decks(card_dictionary=card_dict, date_string=formatted_date)

worldsish_annual_data = pd.DataFrame.from_dict(card_dict,orient="index")
# corp_data = pd.merge(
#     cards_df[cards_df['type_code'] == 'agenda'],
#     trashorbusto_corp_df[trashorbusto_corp_df['rating'] > 1200],
#     how='inner',
#     left_on='code',
#     right_on='nrdb_key'
# )
# corp_data = pd.merge(
#     corp_data,
#     worldsish_annual_data,
#     left_on='code',
#     right_index=True
# )
# corp_data.to_csv("CorpCardsDecklistFrequency.csv")
# corp_data = pd.read_csv("CorpCardsDecklistFrequency.csv")
# corp_data['copies_per_deck'] = corp_data['count']/corp_data['decklists']
# corp_data[['faction_code','title','rating','count','decklists','copies_per_deck']]
worldsish_annual_data

corp_data = pd.merge(
    cards_df,
    trashorbusto_corp_df[trashorbusto_corp_df['rating'] > 1200],
    how='inner',
    left_on='code',
    right_on='nrdb_key'
)
corp_data = pd.merge(
    corp_data,
    worldsish_annual_data,
    left_on='code',
    right_index=True
)

corp_data.to_csv("corp_data_torb_and_counts.csv")
corp_data[['faction','title','ranking','count','decklists','cost','strength','keywords']]

# display unique type_code values
corp_types = ['agenda','asset','operation','upgrade', 'ice']
runner_types = ['event','hardware','resource','program']


df = pd.read_csv("corp_data_torb_and_counts.csv", index_col=0)
df[['faction_code','faction_cost','title', 'type_code','rating','count','decklists','cost','strength','keywords']].sort_values("count", ascending=False).head(30)

counts_df = pd.DataFrame.from_dict(card_dict,orient="index")
merged_data = pd.merge(
    cards_df,
    counts_df,
    left_on='code',
    right_index=True)
merged_data.columns
merged_data[['code','title','count','decklists', "type_code"]].groupby('title').agg({'count':'sum','decklists':'sum'}).sort_values('count',ascending=False)